## Working Version - Container Transfer Hub

**Example description:** Example of a container transfer hub, where very large container vessels deliver containers, while smaller vessels take care of the distribution to the hinterland.

* [**0. Import required libraries:**](#0.-Import-required-libraries)<br>
* [**1. Define work method:**](#1.-Define-work-method)<br>
   * [**1.1 Sites:**](#1.1-Define-the-project-sites)<br>
   * [**1.2 Equipment:**](#1.2-Define-the-project-equipment)<br>
   * [**1.3 Activities:**](#1.3-Define-the-activity)<br>
* [**2. Run the simulation:**](#2.-Run-the-simulation)<br>
* [**3. Post processing:**](#3.-Postprocessing)<br>

## 0. Import required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style
import pyproj

# package(s) for data handling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# digital twin package
import digital_twin.core as core
import digital_twin.model as model
import digital_twin.plot as plot

# Used for making the graph to visualize our problem
import networkx as nx

# Used for mathematical functions
import math

# transport network analysis package
# import transport_network_analysis.core as core
# import transport_network_analysis.model as model

# Vessel database
location_vessel_database = "richtlijnen-duwbakken-2017.csv"

# Water depth on Rhine branches

Results_WaterDepth = "Results20-5.csv"

In [2]:
# Create simulation environment
simulation_start = datetime.datetime(2019, 1, 1)

my_env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

my_env.epoch = time.mktime(simulation_start.timetuple())

## 1. Define work method

### 1.1 Define the project sites
You can specify a project site object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.Locatable - enables you to add coordinates to extract distance information and visualize
    core.HasContainer - enables you to add information on the material available at the site
    core.HasResource - enables you to add information on serving equipment
    core.HasWeather - enables you to add weather conditions

#### Create objects and specify the properties for sites you wish to create

In [29]:
# Importeren van SOBEK output

water_depth = pd.read_csv(Results_WaterDepth, parse_dates=[0], decimal=',').head(100)
print(water_depth)



# type(water_depth.loc[1, 'x'])
# type(water_depth.loc[1, 'y'])
# type(water_depth.loc[1, 'Water depth [m]'])

     time [-]              x              y       branch  chainage  \
0  2085-01-01  204322.000000  429166.000000    Bovenrijn       0.0   
1  2085-01-01  203660.750657  429413.606315    Bovenrijn     706.0   
2  2085-01-01  203166.230068  429733.835638    Bovenrijn    1293.0   
3  2085-01-01  202716.899492  430114.983505    Bovenrijn    1880.0   
4  2085-01-01  202266.888403  430495.645787    Bovenrijn    2467.0   
5  2085-01-01  201789.212487  430839.242525    Bovenrijn    3053.0   
6  2085-01-01  201285.152145  431144.751819    Bovenrijn    3640.0   
7  2085-01-01  200757.942801  431408.203064    Bovenrijn    4227.0   
8  2085-01-01  200215.000000  431638.000000    Bovenrijn    4814.0   
9  2085-01-01  159990.000000  433341.000000        Waal4       0.0   
10 2085-01-01  159515.684807  433189.221904        Waal4     499.0   
11 2085-01-01  159082.980972  432940.559484        Waal4     999.0   
12 2085-01-01  158698.861953  432622.450726        Waal4    1498.0   
13 2085-01-01  15835

In [30]:
# Omzetten van RD naar WGS84 coordinaten
from pyproj import Proj, transform

p1 = Proj(init='epsg:28992')
p2 = Proj(proj='latlong',datum='WGS84')

lon, lat = transform(p1, p2, water_depth['x'].values, water_depth['y'].values)
water_depth['x'], water_depth['y'] = lon, lat
# print(lon, lat)
print(water_depth['x'], water_depth['y'])


0     6.103025
1     6.093463
2     6.086331
3     6.079862
4     6.073382
5     6.066495
6     6.059219
7     6.051599
8     6.043747
9     5.459687
10    5.452795
11    5.446507
12    5.440924
13    5.435876
14    5.431418
15    5.427498
16    5.424040
17    5.421055
18    5.418556
19    5.416555
20    5.414569
21    5.411670
22    5.407888
23    5.403285
24    5.397972
25    5.392642
26    5.387790
27    5.383450
28    5.379677
29    5.376462
        ...   
70    5.137385
71    5.133573
72    5.133410
73    5.129093
74    5.129093
75    5.123937
76    5.118009
77    5.111272
78    5.104098
79    6.103882
80    6.104265
81    6.100454
82    6.096615
83    6.093847
84    6.088459
85    6.081303
86    6.077491
87    6.078720
88    5.844796
89    5.837517
90    5.830126
91    5.822695
92    5.815251
93    5.807821
94    5.800376
95    5.792943
96    5.785513
97    5.778066
98    5.844796
99    5.838219
Name: x, Length: 100, dtype: float64 0     51.848884
1     51.851168
2     51.854089


In [31]:
def connect_sites_with_path(data_from_site, data_to_site, data_node, path):
    Nodes = []
    Edges = []
    Site = type('Site', (core.Identifiable, # Give it a name
             core.Log,          # Allow logging of all discrete events
             core.Locatable,    # Add coordinates to extract distance information and visualize
             core.HasContainer, # Add information on the material available at the site
             core.HasResource), # Add information on serving equipment
    {})                               # The dictionary is empty because the site type is generic

    Node = type('Node', (core.Identifiable, # Give it a name
             core.Log,                      # Allow logging of all discrete events
             core.Locatable),               # Add coordinates to extract distance information and visualize
    {})                                           # The dictionary is empty because the site type is generic

    for i, j in enumerate(path):
        if i == 0:
            data_from_site["geometry"]=shapely.geometry.Point(path[i][0], path[i][1])
            Nodes.append(Site(**data_from_site))

        elif i == len(path) - 1:
            data_to_site["geometry"]=shapely.geometry.Point(path[i][0], path[i][1])
            Nodes.append(Site(**data_to_site))
            Edges.append([Nodes[i-1], Nodes[i]])
            
        else:
            data_node["geometry"]=shapely.geometry.Point(path[i][0], path[i][1])
            data_node["name"]='node-' + str(i)
            node = Node(**data_node)
            node.name = node.id
            Nodes.append(node)
            Edges.append([Nodes[i-1], Nodes[i]])
            
    return Nodes, Edges


In [32]:
data_from_site = {"env": my_env,                  # The simpy environment defined in the first cel
                  "name": "Origin locatie",       # The name of the site
                  "geometry": [],                 # The coordinates of the project site
                  "capacity": 10_000_000,         # The capacity of the site
                  "level": 10_000_000}            # The actual volume of the site

data_inter_site = {"env": my_env,                 # The simpy environment defined in the first cel
                "name": "Overslag locatie",       # The name of the site
                "geometry": [],                   # The coordinates of the project site
                "capacity": 100_000_000,          # The capacity of the site
                "level": 0}                       # The actual volume of the site (empty of course)

data_to_site1 = {"env": my_env,                   # The simpy environment defined in the first cel
                "name": "Haven Nijmegen",         # The name of the site
                "geometry": [],                   # The coordinates of the project site
                "capacity": 5_000_000,            # The capacity of the site
                "level": 0}                       # The actual volume of the site (empty of course)

# data_to_site2 = {"env": my_env,                   # The simpy environment defined in the first cel
#                 "name": "Haven Deventer",         # The name of the site
#                 "geometry": [],                   # The coordinates of the project site
#                 "capacity": 5_000_000,            # The capacity of the site
#                 "level": 0}                       # The actual volume of the site (empty of course)

data_node = {"env": my_env,                       # The simpy environment defined in the first cel
                 "name": "Intermediate site",     # The name of the waterway
                 "geometry": []}                  # The coordinates of the project site


In [33]:
# a = 5.5
# type(a)
# b = int(a)
# print(b)
# type(b)


In [34]:
# Laat een nieuw lijstje met waterstanden maken iedere keer dat chainage op 0 staat. Dit signaleert een nieuw stuk rivier.

# df = water_depth.iloc[:, :]
# for index, row in water_depth.iterrows():
#     if row['chainage'] == 0:
        
#         df[index] = water_depth.loc[index:, :]
        
# print(df[index])


# for i in water_depth['chainage']:
#     if i == 0

# river_branches = []

# for index, row in water_depth.iterrows():
#     if row['chainage'] == 0:
#         river_branches.append(row['branch'])
#     real_water_depth = pd.Series.tolist(water_depth['Water depth [m]'])
#     keys = river_branches
#     vals = real_water_depth
#     data = {key:vals[n] for n, key in enumerate(keys)}

# data = {}

# for index, row in water_depth.iterrows():
#     if row['chainage'] == 0:
#         info = {row['branch']: row['Water depth [m]']}
#         data.update(info)
#     else:
#         info = {row['branch']: row['Water depth [m]']} 
#         data.update(info)

# print(data)

# print(river_branches[:])

# print(water_depth)

     time [-]         x          y       branch  chainage  Water depth [m]
0  2085-01-01  6.103025  51.848884    Bovenrijn       0.0         6.623480
1  2085-01-01  6.093463  51.851168    Bovenrijn     706.0         6.859350
2  2085-01-01  6.086331  51.854089    Bovenrijn    1293.0         6.885520
3  2085-01-01  6.079862  51.857553    Bovenrijn    1880.0         6.850290
4  2085-01-01  6.073382  51.861013    Bovenrijn    2467.0         7.429560
5  2085-01-01  6.066495  51.864141    Bovenrijn    3053.0         7.062410
6  2085-01-01  6.059219  51.866929    Bovenrijn    3640.0         7.884900
7  2085-01-01  6.051599  51.869341    Bovenrijn    4227.0         7.964230
8  2085-01-01  6.043747  51.871451    Bovenrijn    4814.0         5.966630
9  2085-01-01  5.459687  51.888571        Waal4       0.0         6.510900
10 2085-01-01  5.452795  51.887211        Waal4     499.0         6.833750
11 2085-01-01  5.446507  51.884979        Waal4     999.0         6.340790
12 2085-01-01  5.440924  

In [35]:
# Makes lists for the route on different branches. Puts all in one list which makes it difficult to use.

path_collection = []
path_inter_to_destination = []

for index, row in water_depth.iterrows():
    if row['chainage'] == 0:
        path_collection.append(path_inter_to_destination)
        path_collection.append(row['branch']) 
        path_inter_to_destination = [[row['x'], row['y']]]
    else:
        path_inter_to_destination.append([row['x'], row['y']])
path_collection.append(path_inter_to_destination)               #zodat de coordinaten van de laatste branch ook mee gaan

path_collection.pop(0) #verwijder het eerste (lege) element in path_collection

for i in range(len(path_collection)):
    if i % 2 != 0:  
        print(path_collection[i-1], path_collection[i])


Bovenrijn [[6.103024526784044, 51.848884222051105], [6.093463270525033, 51.851167788676584], [6.086331026767716, 51.85408890892047], [6.079862208705836, 51.85755325076327], [6.0733824495897615, 51.86101292984545], [6.066495046679919, 51.86414143843604], [6.059218518102545, 51.866929414087345], [6.051599403794567, 51.869340831792584], [6.043746612091993, 51.871450554365104]]
Waal4 [[5.459686699782352, 51.88857132164646], [5.452794861614324, 51.887211156099696], [5.446506693775746, 51.88497945734284], [5.440924047879587, 51.88212289571123], [5.435875740533783, 51.87890799145611], [5.431417530419897, 51.875374848939764], [5.427497732897521, 51.8715931177821], [5.424040112957532, 51.867652722206365], [5.421054697510236, 51.86356713812186], [5.418556465378742, 51.85935816991354], [5.416554762557886, 51.85503934276316], [5.4145691787100425, 51.8507289045914], [5.411670290548838, 51.846622624185386], [5.407887739840218, 51.842802185990614], [5.403285391949514, 51.83933263188429], [5.397971737

In [36]:
path_from_to_inter = [[3.223979059949225,52.03276781167011],
                        [3.656045091181079,52.04776907241202],
                        [3.953697524848219,52.02803067898825],
                        [4.067560378249368,51.95173269362142]]

In [ ]:
# path_inter_to_destination1 = [[4.067560378249368,51.95173269362142],
#                              [5.260911,51.820500],
#                              [5.830619709457634,51.86183419049677],
#                              [5.830727947258836,51.86102575658615]]

In [ ]:
# path_inter_to_destination2 = [[4.067560378249368,51.95173269362142],
#                              [5.937599,51.964640],
#                              [6.197050,52.092425],
#                              [6.172119444444445,52.247375]]

#### Finally create specific instances of the predefined objects with the specified properties

In [37]:
# Dataframe naar een lijst gemaakt zodat er mee gerekend kan worden maar die lijst verliest zijn attributes zodat er geen graaf 
# van gemaakt kan worden.
# Niet handig. Begrijp niet hoe ik het dataframe handiger kan splitten op chainage en branch.

# Nodes1, Edges1 = connect_sites_with_path(data_from_site, data_inter_site, data_node, path_from_to_inter)
# Nodes1
Nodes2=[]
Edges2=[]

for k in range(len(path_collection)):
    if k % 2 != 0:
        Nodes2.append(connect_sites_with_path(data_inter_site, data_to_site1, data_node, path_collection[k])[0])
        Edges2.append(connect_sites_with_path(data_inter_site, data_to_site1, data_node, path_collection[k])[1])
        
print(len(Nodes2))
print(len(Edges2))
print(Nodes2[0])

# # Nodes3, Edges3 = connect_sites_with_path(data_inter_site, data_to_site2, data_node, path_inter_to_destination2)
# # print(connect_sites_with_path(data_from_site, data_to_site1, data_node, path))

7
7
[<__main__.Site object at 0x000002070037F470>, <__main__.Node object at 0x000002077BFEAEF0>, <__main__.Node object at 0x000002077BFEA6A0>, <__main__.Node object at 0x000002077BFEA4E0>, <__main__.Node object at 0x000002077BFEAA58>, <__main__.Node object at 0x000002077BFEAF60>, <__main__.Node object at 0x000002077BFEA5F8>, <__main__.Node object at 0x000002077BFEA5C0>, <__main__.Site object at 0x000002077BFEA4A8>]


#### Create graph

In [38]:
def add_path_to_graph(G, positions, Nodes, Edges):
    
    for node in Nodes:
        print(node.name)
        positions[node.name] = (node.geometry.x, node.geometry.y)
        G.add_node(node.name, geometry = node.geometry)
        
    for edge in Edges:
        print(edge[0].name + ' - ' + edge[1].name)
        G.add_edge(edge[0].name, edge[1].name, weight = water_depth['chainage'])
        
    return G, positions

positions = {}
G = nx.Graph()

G, positions = add_path_to_graph(G, positions, Nodes2, Edges2)

# for i in range(len(Nodes2)):
# # G, positions = add_path_to_graph(G, positions, Nodes1, Edges1)
#     G, positions = add_path_to_graph(G, positions, Nodes2[i], Edges2[i])
# # G, positions = add_path_to_graph(G, positions, Nodes3, Edges3)

nx.draw(G, positions)

my_env.G = G

AttributeError: 'list' object has no attribute 'name'

In [ ]:
#Koppelen van water levels aan Edges. Dit lijstje wordt op den duur vervangen door een lijst die uit Sobek moet rollen.

# print(Edges2)

avg_water_depth = []

# print(water_depth['Water depth [m]'].values[1])
# print(water_depth['Water depth [m]'].values[1] + water_depth['Water depth [m]'].values[2])
# print(water_depth.loc[1, 'Water depth [m]'] + water_depth.loc[2, 'Water depth [m]'])
# print(float(water_depth.loc[1, 'Water depth [m]']) + float(water_depth.loc[2, 'Water depth [m]']))

#For Nodes2 and Edges2
for i in range(int(len(path_inter_to_destination1)-1)):
        avg_water_depth.append( (water_depth.loc[i, 'Water depth [m]'] + water_depth.loc[i+1, 'Water depth [m]'])* 0.5)
        edge_attribute_dict = {
            (Nodes2[i].name, Nodes2[i+1].name) : {"water level": avg_water_depth[i]},
            }
        nx.set_edge_attributes(G, edge_attribute_dict)


print(G.edges(data=True))

# print(avg_water_depth[:])
# print(range(len(path_inter_to_destination1)))
# len(path_inter_to_destination1)

### 1.2 Define the project equipment
You can specify a vessel object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.ContainerDependentMovable - A moving container, so capacity and location
    core.Processor - Allow for loading and unloading
    core.HasResource - Add information on serving equipment
    core.HasDepthRestriction - Add information on depth restriction 

#### First create objects with the desired properties

In [ ]:
vessel_db = pd.read_csv(location_vessel_database)
print(vessel_db)
# vessel_db.head(10)
# print(len(vessel_db.index))
# print(range(len(vessel_db.index)))
# vessel_db
# print(vessel_db.keys())

In [ ]:
Vessel = type('Vessel',
              (core.Identifiable, 
               core.Log, 
               core.ContainerDependentMovable,
               core.Processor, 
               core.HasResource, 
               core.Routeable,
               core.VesselProperties), 
              {})

In [ ]:
# generator = model.VesselGenerator(Vessel, vessel_db)
# print(generator)
# print(Vessel.VesselProperties)

#### Next specify the properties for vessel(s) you wish to create

In [ ]:
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda filling: filling * (v_full - v_empty) + v_empty

def compute_loading(rate):
    return lambda current_level, desired_level: (desired_level - current_level) / rate

def compute_unloading(rate):
    return lambda current_level, desired_level: (current_level - desired_level) / rate

In [ ]:
# Draught of the vessel is dependent on the degree of filling
def draught_vessel(draught_empty, draught_full):
    return lambda filling: filling * (draught_full - draught_empty) + draught_empty


In [ ]:
for line in vessel_db.index:
#   print(line)
#   print(vessel_db['vessel_type'][line], vessel_db[vessel_db.keys()[0]][line])
#     if line <= 7: #aanzetten wanneer alleen kleinere barges
        vessel_stats[line] = {"env": my_env,
               "name": vessel_db.iloc[line, 0],
               "geometry": Nodes1[-1].geometry,
               "loading_func": compute_loading(1.5),
               "unloading_func": compute_unloading(1.5),
               "compute_v": compute_v_provider(5, 4.5),
               "vesselWidth": vessel_db.iloc[line, 1],
               "vesselLength": vessel_db.iloc[line, 2],
               "vesselDraught_Empty": vessel_db.iloc[line, 3],
               "vesselDraught_Full": vessel_db.iloc[line, 4],
               "capacity": vessel_db.iloc[line, 5],
               }
#         print(vessel_stats[line])
# print(vessel_stats[0].get('name')) #werkt


In [ ]:
# Large bulk vessel variables
data_bulk_large = {"env": my_env,                              # The simpy environment 
               "name": "Bulk large",                           # Name
               "geometry": Nodes1[0].geometry,                 # It starts at the "from site"
               "loading_func": compute_loading(1.5),           # Loading rate
               "unloading_func": compute_unloading(1.5),       # Unloading rate
               "compute_v": compute_v_provider(5, 4.5),        # Sailing speed empty and full
               "vesselWidth": 100,                             # Vessel width
               "vesselLength": 200,                            # Vessel length
               "vesselDraught_Empty": 5,                       # Vessel draught empty
               "vesselDraught_Full": 5,                        # Vessel draught full
               "capacity": 100000,                             # Capacity in tons
               }

In [ ]:
# # Small container vessel variables
# data_container_small_01 = {"env": my_env,                           # The simpy environment 
#                "name": "Container small_01",                        # Name
#                "geometry": Nodes1[-1].geometry,                     # It starts at the "inter site"
#                "loading_func": compute_loading(1.5),                # Loading rate
#                "unloading_func": compute_unloading(1.5),            # Unloading rate
#                "capacity": 5_000,                                   # Capacity of the container vessel - TEU
#                "compute_v": compute_v_provider(5, 4.5),             # Variable speed
#                "compute_draught": draught_vessel(1, 2),             # Variable draught of the vessel
#                 }

In [ ]:
# # Small container vessel variables
# data_container_small_02 = {"env": my_env,                           # The simpy environment 
#                "name": "Container small_02",                        # Name
#                "geometry": Nodes1[-1].geometry,                     # It starts at the "inter site"
#                "loading_func": compute_loading(1.5),                # Loading rate
#                "unloading_func": compute_unloading(1.5),            # Unloading rate
#                "capacity": 5_000,                                   # Capacity of the container vessel - TEU
#                "compute_v": compute_v_provider(5, 4.5),             # Variable speed
#                "compute_draught": draught_vessel(1, 2),             # Variable draught of the vessel
#                 }

#### Finally create specific instances of the predefined objects with the specified properties

In [ ]:
# The simulation object
barge_large = Vessel(**data_bulk_large)
barge_01 = Vessel(**vessel_stats[0])
barge_02 = Vessel(**vessel_stats[1])
barge_03 = Vessel(**vessel_stats[2])
barge_04 = Vessel(**vessel_stats[3])
barge_05 = Vessel(**vessel_stats[4])
barge_06 = Vessel(**vessel_stats[5])
barge_07 = Vessel(**vessel_stats[6])
barge_08 = Vessel(**vessel_stats[7])
barge_09 = Vessel(**vessel_stats[8])
barge_10 = Vessel(**vessel_stats[9])

#Misschien beter om voorlopig een gemiddeld vessel te gebruiken? Kan dan niet vessel afhankelijk de diepte bekijken wat onhandig is.


In [ ]:
# def start(my_env, vessel):
#     while True:
        
#         vessel.log_entry("Start sailing", my_env.now, "0", vessel.geometry)
#         yield from vessel.move()
#         vessel.log_entry("Stop sailing", my_env.now, "0", vessel.geometry)

#         if vessel.geometry == nx.get_node_attributes(my_env.G, "geometry")[vessel.route[-1]]:
#             break

In [ ]:
# vessels = []

# # random_1 = random.choice(list(G))
# # print(random_1)

# # Add 10 vessels to the simulation
# for i in range(100):
#     path = nx.dijkstra_path(G, Nodes1[0].name, Nodes1[-1].name)

    
#     vessel = generator.generate(my_env, "Vessel " + str(i))
#     vessel.route = path
# #   vessel.compute_v = compute_v_provider(5, 4.5)
#     vessel.geometry = nx.get_node_attributes(my_env.G, "geometry")[vessel.route[0]]
#     vessels.append(vessel)
    
#     # Add the movements of the vessel to the simulation
#     env.process(start(my_env, vessel))

### 1.3 Define the activity

In [ ]:
# # conditie dat intermediate niet leeg is
# condition = model.LevelCondition(container=intermediate.container, min_level=1)  

# # conditie dat een jaar verstreken is
# stop_condition = modelTimeCondition(…)  
 
# activity_2 = model.Activity(…, 
# start_condition = None,  # laat de activiteit meteen starten als de simulatie begint
# condition = condition,  # steeds als tussendoor de intermediate container leeg is, wacht een uur en check daarna nog eens
# stop_condition = stop_condition)  # de activiteit eindigt als een jaar verstreken is
# (idem voor activity_3)


In [ ]:
# Stop events:

stop_event_1 = my_env.timeout(365 * 24 * 3600)   # One year of simulation
stop_event_2 = Nodes1[0].container.empty_event   # Or until origin is empty

stop_event = [stop_event_1, stop_event_2]

In [ ]:
# Create activity
activity_1 = model.Activity(env = my_env,                   # The simpy environment defined in the first cel
                          name = "International transport", # We are moving soil
                          origin = Nodes1[0],               # We originate from the from_site
                          destination = Nodes1[-1],         # And therefore travel to the to_site
                          loader = barge_large,         # The benefit of a TSHD, all steps can be done
                          mover = barge_large,          # The benefit of a TSHD, all steps can be done
                          unloader = barge_large,       # The benefit of a TSHD, all steps can be done
                          stop_event = stop_event)          # Stop after 1 year simulation or origin = empty

In [ ]:
# Stop events:

stop_event_1 = my_env.timeout(365 * 24 * 3600)   # One year of simulation
stop_event_2 = Nodes2[-1].container.full_event   # Or until origin is empty

stop_event1 = [stop_event_1, stop_event_2]

In [ ]:
# Create activity                      
activity_2 = model.Activity(env = my_env,                   # The simpy environment defined in the first cel
                          name = "Hinterland transport Nijmegen",    # We are moving soil
                          origin = Nodes1[-1],              # We originate from the from_site
                          destination = Nodes2[-1],         # And therefore travel to the to_site
                          loader = barge_01,      # The benefit of a TSHD, all steps can be done
                          mover = barge_01,       # The benefit of a TSHD, all steps can be done
                          unloader = barge_01,    # The benefit of a TSHD, all steps can be done
                          stop_event = stop_event1)            # Stop after 1 year simulation

In [ ]:
# Stop events:

stop_event_1 = my_env.timeout(365 * 24 * 3600)   # One year of simulation
stop_event_2 = Nodes3[-1].container.full_event   # Or until origin is empty

stop_event2 = [stop_event_1, stop_event_2]

In [ ]:
# Create activity                      
activity_3 = model.Activity(env = my_env,                   # The simpy environment defined in the first cel
                          name = "Hinterland transport Deventer",    # We are moving soil
                          origin = Nodes1[-1],              # We originate from the from_site
                          destination = Nodes3[-1],         # And therefore travel to the to_site
                          loader = barge_02,      # The benefit of a TSHD, all steps can be done
                          mover = barge_02,       # The benefit of a TSHD, all steps can be done
                          unloader = barge_02,    # The benefit of a TSHD, all steps can be done
                          stop_event = stop_event2)            # Stop after 1 year simulation

### 2. Run the simulation

In [ ]:
my_env.run()

print("\n*** Transportation of containers finished in {} ***".format(datetime.timedelta(seconds=int(my_env.now - my_env.epoch))))

### 3. Postprocessing

In [ ]:
vessels = [barge_large, barge_01, barge_02]
# print(vessels)

# import pandas as pd
# df = pd.DataFrame.from_dict(vessels[0].log)
# df

In [ ]:
Nodes1[0].container.level

In [ ]:
Nodes1[-1].container.level

In [ ]:
for j in Nodes2[-1].log['Value']:
    while Nodes2[-1].log['Value'][j] < Nodes2[-1].container.capacity:
        j = j + 1
    else:
        print('Capacity of Nijmegen port is reached on', Nodes2[-1].log['Timestamp'][j])
        print('Capacity of Nijmegen port is reached after', Nodes2[-1].log['Timestamp'][j] - (datetime.datetime(1970, 1, 1, 1, 0, 0) + datetime.timedelta(seconds=my_env.epoch)))
        break

In [ ]:
for j in Nodes3[-1].log['Value']:
    while Nodes3[-1].log['Value'][j] < Nodes2[-1].container.capacity:
        j = j + 1
    else:
        print('Capacity of Deventer port is reached on', Nodes3[-1].log['Timestamp'][j])
        print('Capacity of Deventer port is reached after', Nodes3[-1].log['Timestamp'][j] - (datetime.datetime(1970, 1, 1, 1, 0, 0) + datetime.timedelta(seconds=my_env.epoch)))
        break

#### Vessel planning

In [ ]:
activities = ['loading', 'unloading', 'sailing filled', 'sailing empty']
colors = {0:'rgb(55,126,184)', 1:'rgb(255,150,0)', 2:'rgb(98, 192, 122)', 3:'rgb(98, 141, 122)'}

plot.vessel_planning(vessels, activities, colors)

#### KML visualisation

In [ ]:
# plot.vessel_kml(my_env, vessels, stepsize = 10000)

In [ ]:
# plot.site_kml(my_env, [Nodes1[0], Nodes1[-1], Nodes2[-1], Nodes3[-1]], stepsize = 10000)

In [ ]:
# #Open the file
# if platform.system():
#     !start ./vessel_movements.kml
#     !start ./site_development.kml
# else:
#     !start explorer ./vessel_movements.kml
#     !start explorer ./site_development.kml

In [ ]:
# Properties toewijzen aan Edges
# for edge in FG.edges(data = True):
#     print(edge)

In [ ]:
# vessels[0].compute_draught

In [ ]:
# a = Nodes3[-1].log['Timestamp'][j]
# import pytz
# b = datetime.datetime(1970, 1, 1, 1, 0, 0) + datetime.timedelta(seconds=my_env.epoch)
# print(a)
# print(b)
# print(a-b)